In [1]:
import ast
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
!pip install scikit-learn

In [6]:
movie_info=pd.read_csv('tmdb_5000_credits.csv')
movie_detailed_info=pd.read_csv('tmdb_5000_movies.csv')

In [7]:
movie_info.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [8]:
movie_detailed_info.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [9]:
merged_df = pd.merge(left=movie_info, right=movie_detailed_info, left_on='movie_id', right_on='id', how='inner')

In [10]:
merged_df.shape[0] #total number of entries in the dataset

4803

In [11]:
merged_df.describe()

,movie_id,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,57165.484281,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,88694.614033,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,5.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,9014.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,14629.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,58610.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,459488.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


#Step-1 : EDA 

1.Basic Preprocessing

In [12]:
merged_df.drop_duplicates(inplace=True)

In [13]:
merged_df[['vote_average', 'vote_count','title_x']].isnull().sum() #no handling required

vote_average    0
vote_count      0
title_x         0
dtype: int64

2.Feature Engineering

In [14]:
merged_df.drop("id",axis=1,inplace=True) #drop unnessasary columns

In [15]:
merged_df[merged_df["title_x"]!=merged_df["title_y"]]

,movie_id,title_x,cast,crew,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,vote_average,vote_count


In [16]:
merged_df.drop("title_y", inplace=True, axis=1)
merged_df.rename(columns={'title_x': 'title'}, inplace=True)

In [17]:
merged_df[merged_df["title"]!=merged_df["original_title"]][["original_language","title","original_title"]].head(5)

,original_language,title,original_title
97,ja,Shin Godzilla,シン・ゴジラ
215,en,Fantastic 4: Rise of the Silver Surfer,4: Rise of the Silver Surfer
235,fr,Asterix at the Olympic Games,Astérix aux Jeux Olympiques
317,zh,The Flowers of War,金陵十三釵
474,fr,Evolution,Évolution


From above two cell output we can see "original_title" column will not help much in our analysis as we will only handle english language, so we can drop this column. also "homepage" column does not add anything extra in the analysis so we will drop this one too.

In [18]:
merged_df.drop(columns=["original_title","homepage"],inplace=True, axis=1)

In [19]:
movies_df=merged_df.copy()

We have a few columns containing dictionaries, which are difficult to process. Therefore, we will identify the top 10 relevant data entries from the specified column and repopulate them. For future reference, we will refrain from modifying anything in the existing dataframe but will instead create a new dataframe.

>cast
>crew>genres
>keywords
>production_companies
>production_countries
>spoken_languages


In [20]:
movies_df=merged_df.copy()

In [21]:
ast.literal_eval(movies_df["spoken_languages"][0]) #run this cell for all the columns mentioned above

[{'iso_639_1': 'en', 'name': 'English'},
 {'iso_639_1': 'es', 'name': 'Español'}]

In [23]:
#run this cell for all the columns mentioned above
list(set([i["name"] for i in ast.literal_eval(movies_df["spoken_languages"][0])][:10]))

['English', 'Español']

In [24]:
#write function to implement above change
def get_first_10_names(row):
    row_list = ast.literal_eval(row)
    first_10_unique_names = set([i["name"] for i in row_list][:10])
    first_10_names = ", ".join(f"{name.replace(' ', '')}" for name in first_10_unique_names)
    return first_10_names

# Apply the function to the column containing lists of dictionaries
movies_df['cast'] = movies_df['cast'].apply(get_first_10_names)
movies_df['crew'] = movies_df['crew'].apply(get_first_10_names)
movies_df['genres'] = movies_df['genres'].apply(get_first_10_names)
movies_df['keywords'] = movies_df['keywords'].apply(get_first_10_names)
movies_df['spoken_languages'] = movies_df['spoken_languages'].apply(get_first_10_names)
movies_df['production_companies'] = movies_df['production_companies'].apply(get_first_10_names)
movies_df['production_countries'] = movies_df['production_countries'].apply(get_first_10_names)

In [26]:
movies_df.head(3)

,movie_id,title,cast,crew,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,19995,Avatar,"SamWorthington, WesStudi, JoelDavidMoore, LazA...","MaliFinn, StephenE.Rivkin, ChristopherBoyes, J...",237000000,"Adventure, Fantasy, Action, ScienceFiction","society, spacecolony, cultureclash, romance, s...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"LightstormEntertainment, IngeniousFilmPartners...","UnitedStatesofAmerica, UnitedKingdom",2009-12-10,2787965087,162.0,"English, Español",Released,Enter the World of Pandora.,7.2,11800
1,285,Pirates of the Caribbean: At World's End,"JackDavenport, KeiraKnightley, ChowYun-fat, Bi...","HansZimmer, GoreVerbinski, MikeStenson, Stephe...",300000000,"Adventure, Fantasy, Action","traitor, loveofone'slife, ocean, alliance, exo...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,"JerryBruckheimerFilms, SecondMateProductions, ...",UnitedStatesofAmerica,2007-05-19,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",6.9,4500
2,206647,Spectre,"ChristophWaltz, DanielCraig, DaveBautista, And...","LeeSmith, BillBernstein, DanielCraig, DennisGa...",245000000,"Adventure, Crime, Action","unitedkingdom, basedonnovel, sequel, spy, mi6,...",en,A cryptic message from Bond’s past sends him o...,107.376788,"Danjaq, B24, ColumbiaPictures","UnitedStatesofAmerica, UnitedKingdom",2015-10-26,880674609,148.0,"Français, Deutsch, Español, Italiano, English",Released,A Plan No One Escapes,6.3,4466


In [31]:
features = ['cast', 'crew', 'title', 'genres', 'keywords', 'overview']

movies_df[features] = movies_df[features].fillna('')
movies_df['combined_features'] = movies_df['cast'] + ',' + movies_df['crew'] + ',' + movies_df['title'] + ',' \
                                + movies_df['genres'] + ',' +movies_df['keywords'] + ' ' + movies_df['overview']

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['combined_features'])

similarity_scores = linear_kernel(tfidf_matrix, tfidf_matrix)# Compute similarity scores using linear kernel

# Function to recommend movies based on similarity scores
def recommend_movies(movie_title,num):
    
    movie_index = movies_df[movies_df['title'] == movie_title].index[0]# Get the index of the movie title
    movie_similarities = list(enumerate(similarity_scores[movie_index]))
    # Sort the movies based on similarity scores
    movie_similarities = sorted(movie_similarities, key=lambda x: x[1], reverse=True)
    # Recommend top 10 similar movies
    recommended_movies = movie_similarities[1:num+1]  # Exclude the first movie (self)
    recommended_movie_titles = [movies_df.iloc[movie[0]]['title'] for movie in recommended_movies]
    
    return recommended_movie_titles

In [32]:
recommend_movies('GoodFellas',10)

['Casino',
 'The Departed',
 'Run All Night',
 "Angela's Ashes",
 'Gangs of New York',
 'Raging Bull',
 'The Wind That Shakes the Barley',
 'This Thing of Ours',
 'Lucky Break',
 'The Age of Innocence']